In [2]:
from dotenv import load_dotenv
import os
from langchain_community.graphs import Neo4jGraph


from langchain_community.graphs import Neo4jGraph

load_dotenv()

AURA_INSTANCENAME = os.environ["AURA_INSTANCENAME"]
NEO4J_URI = os.environ["NEO4J_URI"]
NEO4J_USERNAME = os.environ["NEO4J_USERNAME"]
NEO4J_PASSWORD = os.environ["NEO4J_PASSWORD"]
NEO4J_DATABASE = os.environ["NEO4J_DATABASE"]
AUTH = (NEO4J_USERNAME, NEO4J_PASSWORD)

In [4]:
kg = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE,
)

/var/folders/4_/0706vvm51q9cr35nk1974z6r0000gn/T/ipykernel_77424/3135869052.py:1: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip install -U :class:`~langchain-neo4j` and import as `from :class:`~langchain_neo4j import Neo4jGraph``.
  kg = Neo4jGraph(


In [9]:
kg.query(
    """
    CREATE VECTOR INDEX health_providers_embeddings IF NOT EXISTS
    FOR (hp:HealthcareProvider) ON (hp.comprehensiveEmbedding)
    OPTIONS {
      indexConfig: {
        `vector.dimensions`: 1536,
        `vector.similarity_function`: 'cosine'
      }
    }
    """
)

[]

In [10]:
res = kg.query(
    """
  SHOW VECTOR INDEXES
  """
)
print(res)

[{'id': 2, 'name': 'health_providers_embeddings', 'state': 'ONLINE', 'populationPercent': 100.0, 'type': 'VECTOR', 'entityType': 'NODE', 'labelsOrTypes': ['HealthcareProvider'], 'properties': ['comprehensiveEmbedding'], 'indexProvider': 'vector-2.0', 'owningConstraint': None, 'lastRead': None, 'readCount': None}]


In [ ]:
kg.query(
    """
    MATCH (hp:HealthcareProvider)-[:TREATS]->(p:Patient)
    WHERE hp.bio IS NOT NULL
    WITH hp, genai.vector.encode(
        hp.bio,
        "OpenAI",
        {
          token: $openAiApiKey,
          endpoint: $openAiEndpoint
        }) AS vector
    WITH hp, vector
    WHERE vector IS NOT NULL
    CALL db.create.setNodeVectorProperty(hp, "comprehensiveEmbedding", vector)
    """,
    params={
        "openAiApiKey": OPENAI_API_KEY,
        "openAiEndpoint": OPENAI_ENDPOINT,
    },
)

In [ ]:
from langchain.schema import BaseRetriever, Document
from rank_bm25 import BM25Okapi
from pydantic import PrivateAttr

class CustomBM25Retriever(BaseRetriever):
    _bm25: BM25Okapi = PrivateAttr()
    _docs: list = PrivateAttr()

    def __init__(self, bm25, docs):
        super().__init__()
        self._bm25 = bm25
        self._docs = docs

    def _get_relevant_documents(self, query: str):
        # Default to retrieving top 5 relevant documents
        return [doc for doc, score in self._get_relevant_documents_with_scores(query, 5)]

    def _get_relevant_documents_with_scores(self, query: str, k: int):
        # Tokenize the query
        query_tokens = query.split()

        # Compute BM25 scores
        scores = self._bm25.get_scores(query_tokens)

        # Pair documents with scores and sort by relevance
        scored_docs = sorted(zip(self._docs, scores), key=lambda x: x[1], reverse=True)

        # Return top k documents with scores
        return scored_docs[:k]

# Sample documents
docs = [
    Document(page_content="Quantum computing is a type of computation that harnesses the collective properties of quantum states."),
    Document(page_content="Machine learning involves the use of algorithms that improve over time with data."),
    Document(page_content="Quantum mechanics is a fundamental theory in physics that describes nature at the smallest scales."),
    Document(page_content="Artificial intelligence encompasses machine learning, natural language processing, and robotics."),
]

# Tokenize the documents
tokenized_docs = [doc.page_content.split() for doc in docs]

# Initialize BM25 with custom parameters
bm25 = BM25Okapi(tokenized_docs, k1=1.5, b=0.7)

# Create the custom retriever
bm25_retriever = CustomBM25Retriever(bm25=bm25, docs=docs)

# Test the retriever
query = "What is quantum mechanics?"
top_k = 2  # Get top 2 relevant documents
retrieved_docs_with_scores = bm25_retriever._get_relevant_documents_with_scores(query, top_k)

# Display retrieved documents and their scores
for i, (doc, score) in enumerate(retrieved_docs_with_scores, 1):
    print(f"Document {i} (Score: {score}):\n{doc.page_content}\n")

Document 1 (Score: 0.7997646767679749):
Quantum computing is a type of computation that harnesses the collective properties of quantum states.

Document 2 (Score: 0.0):
Machine learning involves the use of algorithms that improve over time with data.



In [25]:
retrieved_docs

[Document(metadata={}, page_content='Quantum computing is a type of computation that harnesses the collective properties of quantum states.')]